In [1]:
import datetime
import hashlib
import string
from itertools import product

The value returned by a hash function is often called a hash, message digest, hash value, or checksum.

The code below takes the "Hello World" string and prints the HEX digest of that string. hexdigest returns a HEX string representing the hash.

In [2]:
string_to_hash = "Hello World".encode( "utf-8" )
hash_object = hashlib.sha256(string_to_hash)
hex_dig = hash_object.hexdigest()
print(hex_dig)

a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e


In [3]:
print(hashlib.sha256(string_to_hash).hexdigest())

a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e


### Hash Code

In [4]:
class HashCache( object ):

    SALT_APPEND_RIGHT = "right"
    SALT_APPEND_LEFT = "left"
    
    def __init__( self, *args, **kwargs ):
        
        self.string_to_hash_map = {}
        self.cache_hit_count = 0
        print( "Cache initialized at " + str( datetime.datetime.now() ) )
        
    #-- END __init__() method --#

    def get_hash( self, string_to_hash_IN, salt_IN = "", append_salt_to_IN = SALT_APPEND_RIGHT ):

        '''
        Accepts string to hash and optional salt value.  If salt, appends it to the right of the string.
            Then, checks to see if that string is already in the hash map.  If so, retrieves hash of it.
            If not, hashes it and caches the hash.  Returns the hash, else None if there was an error.
        '''

        # return reference
        hash_OUT = ""

        # declare variables
        working_string = ""
        temp_hash = ""

        # pull string into working string:
        working_string = string_to_hash_IN

        # empty?
        if ( ( working_string is not None ) and ( working_string != "" ) and ( working_string != "NaN" ) ):

            # Got a value.  hash it.

            # is there a salt?
            if ( ( salt_IN is not None ) and ( salt_IN != "" ) ):

                # yes - use it.  Append to right or left?
                if ( append_salt_to_IN == self.SALT_APPEND_RIGHT ):

                    # right.
                    working_string = working_string + salt_IN

                else:

                    # if not right, left.
                    working_string = salt_IN + working_string

                #-- END check to see if right or left append --#

            #-- END check to see if salt. --#

            # check to see if hash in cache
            if ( working_string in self.string_to_hash_map ):

                # cached - record cache hit.
                self.cache_hit_count += 1

            else:

                # not cached.  Hash and cache.

                # encode to UTF-8
                temp_hash = working_string.encode( "utf-8" )

                # hash
                temp_hash = hashlib.sha256( temp_hash ).hexdigest()
                
                # cache
                self.string_to_hash_map[ working_string ] = temp_hash

            #-- END check to see if cached. --#

            # retrieve hash from cache.
            hash_OUT = self.string_to_hash_map[ working_string ]

        else:

            # No string in.  Leave empty.
            hash_OUT = ""

        #-- END check to see if empty --#


        return hash_OUT

    #-- END function get_hash() --#
    
#-- END class HashCache --#

# print( "Object HashCache defined at " + str( datetime.datetime.now() ) )


#### Create object of class HashCache

In [5]:
hashCacheObject = HashCache()

Cache initialized at 2019-02-28 13:05:16.819203


In [6]:
hashCacheObject

In [7]:
print("hashCacheObject.cache_hit_count: ", hashCacheObject.cache_hit_count)
print("hashCacheObject.SALT_APPEND_LEFT: ", hashCacheObject.SALT_APPEND_LEFT)
print("hashCacheObject.SALT_APPEND_RIGHT: ", hashCacheObject.SALT_APPEND_RIGHT)
print("hashCacheObject.string_to_hash_map: ", hashCacheObject.string_to_hash_map)

hashCacheObject.cache_hit_count:  0
hashCacheObject.SALT_APPEND_LEFT:  left
hashCacheObject.SALT_APPEND_RIGHT:  right
hashCacheObject.string_to_hash_map:  {}


#### Now let's call the get_hash function and hash some values!

In [8]:
# define the string to be hashed
string_to_hash = "Responsible"

#### For now let's not add any salt

In [9]:
salt = ""
append_salt_to_IN = "right"

In [10]:
# to be able to call the get_hash function from class HashCache, we need to use its object
generated_hash_code = hashCacheObject.get_hash(string_to_hash, salt, append_salt_to_IN)


#### Let's check whether our hash was correct by using an online generator for the same string: https://passwordsgenerator.net/sha256-hash-generator/

In [11]:
generated_hash_online = "BC110A6D0722098AC63E93B5FD6222AFE478819EC5A6F4CE2557330684EB6264"
generated_hash_online = generated_hash_online.lower()
print(generated_hash_online)

bc110a6d0722098ac63e93b5fd6222afe478819ec5a6f4ce2557330684eb6264


In [12]:
generated_hash_code

'bc110a6d0722098ac63e93b5fd6222afe478819ec5a6f4ce2557330684eb6264'

In [13]:
print(generated_hash_code == generated_hash_online)

True


In [14]:
print("hashCacheObject.cache_hit_count: ", hashCacheObject.cache_hit_count)
print("hashCacheObject.SALT_APPEND_LEFT: ", hashCacheObject.SALT_APPEND_LEFT)
print("hashCacheObject.SALT_APPEND_RIGHT: ", hashCacheObject.SALT_APPEND_RIGHT)
print("hashCacheObject.string_to_hash_map: ", hashCacheObject.string_to_hash_map)

hashCacheObject.cache_hit_count:  0
hashCacheObject.SALT_APPEND_LEFT:  left
hashCacheObject.SALT_APPEND_RIGHT:  right
hashCacheObject.string_to_hash_map:  {'Responsible': 'bc110a6d0722098ac63e93b5fd6222afe478819ec5a6f4ce2557330684eb6264'}


## Cracking the hashed values

Hashing is a one-way function. So it's not like we can do a "decrpyt" on them.

The only ways available to do this is using a dictionary or a brute-force method.

### Dictionary Attack
One way to crack hashcodes is by using an external dictionary which consists the common passwords etc used.

Dictionaries come with actual strings, so we need to first hash them using the same hash algorithm that we used above and then do a match of the dictionary hashed word with our hashed value.

The first wordlist that we are going to try here is from here: https://github.com/praetorian-inc/Hob0Rules/blob/master/wordlists/rockyou.txt.gz

There are some other well known dictionaries as well: https://crackstation.net/crackstation-wordlist-password-cracking-dictionary.htm

In [15]:

# Build Empty Dictionary
dictionary = {}
hashList =[]
wordlist = "rockyou.txt"

print("Processing word list into hashed word list...")
# Run through word list, and create hashed dictionary
with open(wordlist, encoding='utf8', errors="ignore") as openedFile:
    for line in openedFile:
        line = line.encode('utf8').rstrip()
        hashed = hashlib.sha256(line).hexdigest()
        dictionary.update({hashed:line})
print("Processed word list into hashed word list...")

hashed_values_to_crack = [generated_hash_code]

print("Iterating through hash list to crack, and comparing against word list...")
# Iterate through word list and compare to hash list

counter=0
for hash in hashed_values_to_crack:
    if hash in dictionary:
        print("*** MATCH- %s %s" % (hash, dictionary.get(hash)))
        counter +=1

print("Processing Complete...")
print("List had:", counter, "matches")


Processing word list into hashed word list...
Processed word list into hashed word list...
Iterating through hash list to crack, and comparing against word list...
Processing Complete...
List had: 0 matches


#### Just to demonstrate that the dictionary attack works if you actually have a matching string

In [16]:
#hashed value of sunshine
hashed_sunshine = "A941A4C4FD0C01CDDEF61B8BE963BF4C1E2B0811C037CE3F1835FDDF6EF6C223".lower()

In [17]:
hashed_sunshine

'a941a4c4fd0c01cddef61b8be963bf4c1e2b0811c037ce3f1835fddf6ef6c223'

In [18]:

# Build Empty Dictionary
# dictionary = {}
# wordlist = "rockyou.txt"

# print("Processing word list into hashed word list...")
# # Run through word list, and create hashed dictionary
# with open(wordlist, encoding='utf8', errors="ignore") as openedFile:
#     for line in openedFile:
#         line = line.encode('utf8').rstrip()
#         hashed = hashlib.sha256(line).hexdigest()
#         dictionary.update({hashed:line})
# print("Processed word list into hashed word list...")

hashed_values_to_crack = [hashed_sunshine]

print("Iterating through hash list to crack, and comparing against word list...")
# Iterate through word list and compare to hash list

counter=0
for hash in hashed_values_to_crack:
    if hash in dictionary:
        print("*** MATCH- %s %s" % (hash, dictionary.get(hash)))
        counter +=1

print("Processing Complete...")
print("List had:", counter, "matches")


Iterating through hash list to crack, and comparing against word list...
*** MATCH- a941a4c4fd0c01cddef61b8be963bf4c1e2b0811c037ce3f1835fddf6ef6c223 b'sunshine'
Processing Complete...
List had: 1 matches


### Brute Force

#### Lets perform a brute force attack for 6 digit license number plates

In [19]:
license_num = 493092
hashed_license_num = "04A29181D728DCBEAFC5151AE6F79D877A2B75D20E0FFA6A7078BF68D8123D67".lower()
print(hashed_license_num)

04a29181d728dcbeafc5151ae6f79d877a2b75d20e0ffa6a7078bf68d8123d67


In [20]:
has_l_num_123456 = '8D969EEF6ECAD3C29A3A629280E686CF0C3F5D5A86AFF3CA12020C923ADC6C92'.lower()

In [21]:
string.digits

'0123456789'

In [22]:
print( list(product([1,2,3],repeat = 2)))

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)]


In [23]:
print( list(product([1,2,3],repeat = 3)))

[(1, 1, 1), (1, 1, 2), (1, 1, 3), (1, 2, 1), (1, 2, 2), (1, 2, 3), (1, 3, 1), (1, 3, 2), (1, 3, 3), (2, 1, 1), (2, 1, 2), (2, 1, 3), (2, 2, 1), (2, 2, 2), (2, 2, 3), (2, 3, 1), (2, 3, 2), (2, 3, 3), (3, 1, 1), (3, 1, 2), (3, 1, 3), (3, 2, 1), (3, 2, 2), (3, 2, 3), (3, 3, 1), (3, 3, 2), (3, 3, 3)]


In [ ]:
# def brute_force():
        
